# Quickstart

## Standard Authentication

### 1. Authenticate with Cromwell using HTTPBasicAuth (username and password)

In [37]:
from cromwell_tools.cromwell_auth import CromwellAuth

auth_httpbasic = CromwellAuth.from_user_password(username='username', 
                                                 password='password', 
                                                 url='https://cromwell.xxx.broadinstitute.org')

### 2. Authenticate with Cromwell using HTTPBasicAuth (secret JSON file)

A secret JSON file should look like the following example with 3 keys:
```JSON
{
    "url": "url",
    "username": "username",
    "password": "password"
}
```

In [3]:
from cromwell_tools.cromwell_auth import CromwellAuth

auth_httpbasic_file = CromwellAuth.from_secrets_file(secrets_file='path/to/secrets_file.json')

### 3. Authenticate with Cromwell using OAuth (service account JSON key file)

In [25]:
from cromwell_tools.cromwell_auth import CromwellAuth

auth_oauth = CromwellAuth.from_service_account_key_file(service_account_key='path/to/service/account/key.json', 
                                                        url='https://cromwell.caas-prod.broadinstitute.org')

### 4. Authenticate with Cromwell with no Auth

In [5]:
# NotImplemented

### 5. [Recommended] Authenticate with Cromwell using the standardized method

In [ ]:
from cromwell_tools.cromwell_auth import CromwellAuth

auth = CromwellAuth.harmonize_credentials(service_account_key='path/to/service/account/key.json', 
                                          url='https://cromwell.caas-prod.broadinstitute.org')

## Submit jobs to Cromwell/Cromwell-as-a-Service

In [35]:
from cromwell_tools import api
from cromwell_tools import utilities

In [38]:
response = api.submit(auth=auth_httpbasic, 
                      wdl_file='Examples/hello_world.wdl', 
                      inputs_files=['Examples/inputs.json'], 
                      dependencies=['Examples/helloworld.wdl'])

In [39]:
response.json()

{'id': '5eb26c8c-07d9-4f07-b728-4805602318de', 'status': 'Submitted'}

In [40]:
api.wait(workflow_ids=[response.json()['id']], auth=auth_httpbasic)

All workflows succeeded!


## Query jobs in Cromwell/Cromwell-as-a-Service

### Get the workflows

In [2]:
from cromwell_tools import api
from cromwell_tools import utilities

There are a lot of query keys can be used to filter workflows
A complicated query dict could be:
```JSON
custom_query_dict = {
    'label': {
        'cromwell-workflow-id': 'cromwell-xxxxxxxx-xxxx-xxxx-xxxx-xxxxxxxxxxxx',
        'project_shortname': 'Name of a project that trigerred this workflow'
    },
    'status': ['Running', 'Succeeded', 'Aborted', 'Submitted', 'On Hold', 'Failed'],
    'id': 'xxxxxxxx-xxxx-xxxx-xxxx-xxxxxxxxxxxx',
    'additionalQueryResultFields': 'labels',
    'submission': '2018-01-01T00:01:01.410150Z',
    'start': '2018-01-01T01:01:01.410150Z',
    'end': '2018-01-01T02:01:01.410150Z',
    'name': ['SmartSeq2SingleCell', '10xCount'],
    'additionalQueryResultFields': ['labels'],
    'includeSubworkflows': True
}
```
We will just use a very simple example here to query for the workflow we just submitted:

In [41]:
custom_query_dict = {
    'name': 'HelloWorld',
    'id': response.json()['id']
}

response = api.query(query_dict=custom_query_dict, auth=auth_httpbasic)

In [42]:
response.json()['results']

[{'name': 'HelloWorld',
  'id': '5eb26c8c-07d9-4f07-b728-4805602318de',
  'submission': '2018-11-09T16:56:36.052Z',
  'status': 'Succeeded',
  'end': '2018-11-09T16:58:19.640Z',
  'start': '2018-11-09T16:56:41.111Z'}]

## Submit jobs with On Hold in Cromwell/Cromwell-as-a-Service
- Submit a job with "On Hold" status.
- Check the status.
- Release the job.

#### Submit the job

In [35]:
from cromwell_tools import api
from cromwell_tools import utilities

In [43]:
response = api.submit(auth=auth_httpbasic, 
                      wdl_file='Examples/hello_world.wdl', 
                      inputs_files=['Examples/inputs.json'], 
                      dependencies=['Examples/helloworld.wdl'], 
                      on_hold=True)

In [44]:
response.json()

{'id': 'e0061585-168e-4635-aa47-aebe366105bb', 'status': 'On Hold'}

#### Check the status of the job

In [45]:
response = api.status(auth=auth_httpbasic, 
                      uuid=response.json()['id'])
response.json()

{'status': 'On Hold', 'id': 'e0061585-168e-4635-aa47-aebe366105bb'}

#### Release the job

In [46]:
response = api.release_hold(auth=auth_httpbasic, 
                            uuid=response.json()['id'])
response.json()

{'id': 'e0061585-168e-4635-aa47-aebe366105bb', 'status': 'Submitted'}